## **ReadMe**

####*Summary:* This is python code for synchronized image collection for widefield calcium imaging using one TeledynePrime BSI Scientific CMOS camera and one FLIRFlea3 USB3 camera. This code can be adapted to acquire using two FLIR Flea3USB3s and the code is available upon request.


####*Hardware:*  TeledynePrime BSI Scientific CMOS camera, FLIRFlea3 USB3, Thorlabs blueLED, additional Thorlabs LEDs (we used UVLED) to collect calcium and non-calcium signals, respectively. Software/platformsused:

    A.  ImageJ (Micro-Manager2.0.0) to configure Prime BSI (set ROI, exposure time/FPS, binning) andnumber of frames to collect using Multi-Dimensional Acquisition.
    
    B.  SpinView (3.1.0.79)to configure FLIR Flea3 USB3. We matched FPS in SpinView to calculated FPS in Micro-Manager.
    
    C.  WindowsPowershell or other terminal. One tip is to set priority of Micro-Manager to“high priority” in task manager to eliminate frame skipping during acquisition(but note, effectiveness of this strategy may vary based on computer specs). Micro-Managermay be listed as Java(™) Platform SE binary (4)or javaw.exe in task manager.
    
    D.  A local python environment with the python packages: pyspin 1.1.1, numpy 1.21.6,matplotlib 3.7.2, pycromanager 34.6 and pyserial 3.5.
    
    E.   We use google colab (for integrated team code access), connecting to the local runtimevia jupyter lab for each acquisition so that the operations are run on ourlocal machine not on Google’s servers.
    
#####*Important note:* this code includes interaction with a custom-built circuit board designed tocontrol sensory stimulation (e.g. auditory tones) and collect treadmill data.Each line of the code that communicates to the custom serial board isspecifically annotated as such. Although your serial board commands willprobably be different than ours, the order the commands are presented serve as an example.

## **Import libraries**

In [ ]:
import os
import pyspin
import sys
import numpy as np
import time
import threading
import datetime
import matplotlib.pyplot as plt
from pycromanager import Studio
!pip install pycromanager --upgrade
import pycromanager
from pycromanager import Core
from pycromanager import Acquisition, multi_d_acquisition_events
from pycromanager import JavaBackendAcquisition, multi_d_acquisition_events
!pip install pycromanager
!pip install --upgrade --force-reinstall pyserial
!pip install pyserial
import serial
print(serial.__file__)

  Using cached pyserial-3.5-py2.py3-none-any.whl.metadata (1.6 kB)
Using cached pyserial-3.5-py2.py3-none-any.whl (90 kB)
  Attempting uninstall: pyserial
    Found existing installation: pyserial 3.5
    Uninstalling pyserial-3.5:
      Successfully uninstalled pyserial-3.5
C:\Users\fractal_1\anaconda3\envs\my_gpu_env\lib\site-packages\serial\__init__.py


## **MultiCam Acquisition All**

In [ ]:

use_serial = True #interact with a custom serial board
readout = True #save custom serial board log files

DATA_FOLDER = "C:\MyCameraData"
current_time_label = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S.%f')  # Timestamp used for file label
NEW_FOLDER = os.path.join(DATA_FOLDER, "FLIR_BSI_TS_FPS_" + current_time_label[:19])
print("New folder path:", NEW_FOLDER, '\n')

NUM_IMAGES = 60000  # Number of images to grab
FPS_TARGET = 40
FRAME_INTERVAL = 1.0 / FPS_TARGET  # Time for each frame in seconds
fps_l = []
time_stamp_l = []
time_stamp_pc_l = []
sync_checkpoints = []

capture_start_event = threading.Event()

#tie PC date and time to elapsed time (nanoseconds)
import datetime
current_datetime = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S.%f')
current_time_ns = time.perf_counter_ns()
sync_checkpoints.append(["Start of script", current_datetime, current_time_ns])
print("Current date and time:", current_datetime)
print("High-precision time (in nanoseconds):", current_time_ns)
#§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•
#§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•

# Define serial port to custom circuit board
if use_serial:
    try: #if not open
        s = serial.Serial(
            port='COM6',         # Replace with your port name
            baudrate=115200,     # Set baud rate to 115200
            parity=serial.PARITY_NONE, # Set parity to None (N)
            stopbits=serial.STOPBITS_ONE, # Set stop bits to 1
            bytesize=serial.EIGHTBITS,   # Set byte size to 8
            write_timeout=5                    # Set a read timeout (in seconds)
            )
    except Exception as e: #if already open (close and define s again)
        print(f"Error: {e}")
        s.close()
        s = serial.Serial(
            port='COM6',         # Replace with your port name
            baudrate=115200,     # Set baud rate to 115200
            parity=serial.PARITY_NONE, # Set parity to None (N)
            stopbits=serial.STOPBITS_ONE, # Set stop bits to 1
            bytesize=serial.EIGHTBITS,   # Set byte size to 8
            write_timeout=5                    # Set a read timeout (in seconds)
            )
    if s.isOpen():
        print("Serial port is open and configured.")
    else:
        print("Failed to open serial port.")
#§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•
#§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•

def set_frame_rate(cam, frame_rate):
    try:
        nodemap = cam.GetNodeMap()
        # Disable automatic frame rate control
        acquisition_frame_rate_auto = pyspin.CEnumerationPtr(nodemap.GetNode("AcquisitionFrameRateAuto"))
        if pyspin.IsAvailable(acquisition_frame_rate_auto) and pyspin.IsWritable(acquisition_frame_rate_auto):
            acquisition_frame_rate_auto_off = acquisition_frame_rate_auto.GetEntryByName("Off")
            if pyspin.IsAvailable(acquisition_frame_rate_auto_off) and pyspin.IsReadable(acquisition_frame_rate_auto_off):
                acquisition_frame_rate_auto.SetIntValue(acquisition_frame_rate_auto_off.GetValue())
        # Set the frame rate
        acquisition_frame_rate = pyspin.CFloatPtr(nodemap.GetNode("AcquisitionFrameRate"))
        if pyspin.IsAvailable(acquisition_frame_rate) and pyspin.IsWritable(acquisition_frame_rate):
            acquisition_frame_rate.SetValue(frame_rate)
    except pyspin.SpinnakerException as ex:
        print("Error: %s" % ex)
#§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•
#§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•

def acquire_images(cam, cam_num, cam_list):
    #define save locations
    image_folder = os.path.join(NEW_FOLDER, f"Camera_{cam_num}_Images")
    os.makedirs(image_folder, exist_ok=True)
    timestamp_file_path = os.path.join(NEW_FOLDER, f"Camera_{cam_num}_Timestamps_{current_time_label}.txt")
    timestamp_file = open(timestamp_file_path, "w")

    print(f'*** IMAGE ACQUISITION FOR CAMERA {cam_num} ***\n')
    try:
        result = True

        # Prepare camera to acquire images
        nodemap = cam.GetNodeMap()
        node_acquisition_mode = pyspin.CEnumerationPtr(nodemap.GetNode('AcquisitionMode'))
        node_acquisition_mode_continuous = node_acquisition_mode.GetEntryByName('Continuous')
        acquisition_mode_continuous = node_acquisition_mode_continuous.GetValue()
        node_acquisition_mode.SetIntValue(acquisition_mode_continuous)

        #start acquisition and record time
        cam.BeginAcquisition()
        print(f'Camera {cam_num} started acquiring images...')


        #¶•¶•¶•¶•¶•¶ Main acquisition loop ¶•¶•¶•¶•¶•¶
        for n in range(NUM_IMAGES):
            start_time = time.perf_counter_ns() #start of frame in loop

            if use_serial:
                #read current buffer and send GO
                if s.in_waiting > 0:
                    dateandtime = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S.%f') #save time of serial data readout ((limited by when s read by script))
                    current_time = time.perf_counter_ns()
                    data = s.readline().decode()
                    print("$$$$$$$$$$$$$$$$$$$$data:", data)
                    sync_checkpoints.append([data, dateandtime, current_time])

                    # This if statement interacts with a custom circuit board
                    if ">SYNC!" in str(data):
                        t = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S.%f')
                        current_time = time.perf_counter_ns()
                        frametimec1 = '>GO!' + t[:8] + ':10\n' #########################################################
                        print("Frametime command written in acquire images:", frametimec1)
                        s.write(frametimec1.encode()) #write GO
                        print("++++++++++++++++++++++++++++++++++++++++++++ Wrote GO to serial")
                        sync_checkpoints.append(["Serial GO command sent", t, current_time]) #record go time

                        #wait event
                        dateandtime = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S.%f')
                        current_time = time.perf_counter_ns()
                        capture_start_event.wait() #wait for all cameras
                        sync_checkpoints.append(["Acquire_images flir capture_start_event.wait()", dateandtime, current_time])

                        #read RUNNING
                        if s.in_waiting > 0:
                            dateandtime = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S.%f') #save time of serial data readout ((limited by when s read by script))
                            current_time = time.perf_counter_ns()
                            data = s.readline().decode()
                            print("$$$$$$$$$$$$$$$$$$$$data:", data)
                            sync_checkpoints.append([data, dateandtime, current_time])

            if n == 0:
                print("++++++++++++++++++++++++++++++++++++++++++++ Begin FLIR acquisition")



            #Get image and record time
            dateandtime1 = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S.%f') #date and time (PC time) 1
            timestamp_image_1 = time.perf_counter_ns() #before acquisition time
            image_result = cam.GetNextImage(1000) #image cap
            timestamp_image_2 = time.perf_counter_ns() #after acquisition time
            dateandtime2 = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S.%f') #date and time (PC time) 2
            time_stamp = image_result.GetTimeStamp() #timestamp specific to the image acquisition (camera time)

            if image_result.IsIncomplete():
                print(f'Camera {cam_num}: Image {n} incomplete with image status {image_result.GetImageStatus()}')
            else:
                #image
                filename = os.path.join(image_folder, f'Camera_{cam_num}_Image_{n}.jpg')
                image_result.Save(filename)
                #print(f'Camera {cam_num}: Image {n} saved at {filename}') #COMMENT OUT

                #timestamp
                time_stamp_l.append(time_stamp)
                timestamp_file.write(f"{time_stamp}\n")
                time_stamp_pc_l.append([timestamp_image_1, timestamp_image_2, dateandtime1, dateandtime2])

            image_result.Release()

            #get fps using time lib
            end_time = time.perf_counter_ns()
            actual_fps = (1.0 / (end_time - start_time))*1e9
            if n < 20:
                print(f'Camera {cam_num}: Frame {n} captured at {actual_fps:.2f} FPS')
            elif n % 1000 == 0:
                print(f'Camera {cam_num}: Frame {n} captured at {actual_fps:.2f} FPS')
            fps_l.append(actual_fps)
        cam.EndAcquisition()

    except pyspin.SpinnakerException as ex:
        print(f'Error: {ex}')
        result = False
    finally:
        print("End acquisition (FLIR)")
        print("Date and time", datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S.%f'))
        # End acquisition if it hasn't been stopped yet
        if cam.IsStreaming():
            cam.EndAcquisition()
    return result
#§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•
#§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•

def print_device_info(nodemap, cam_num):
    print('Printing device information for camera %d... \n' % cam_num)
    try:
        result = True
        node_device_information = pyspin.CCategoryPtr(nodemap.GetNode('DeviceInformation'))
        if pyspin.IsReadable(node_device_information):
            features = node_device_information.GetFeatures()
            for feature in features:
                node_feature = pyspin.CValuePtr(feature)
                print('%s: %s' % (node_feature.GetName(),
                                  node_feature.ToString() if pyspin.IsReadable(node_feature) else 'Node not readable'))
        else:
            print('Device control information not readable.')
        print()
    except pyspin.SpinnakerException as ex:
        print('Error: %s' % ex)
        return False
    return result
#§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•
#§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•

def run_studio_acquisition():
    core = Core()
    print(core)
    try:
        studio = Studio(convert_camel_case=False)  # Initialize Studio
        capture_start_event.wait() #wait for all cameras
        dateandtime = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S.%f')
        current_time = time.perf_counter_ns()  # Get current time (python)        \
        studio.acquisitions().run_acquisition_nonblocking()  # Start acquisition with Studio
        print("++++++++++++++++++++++++++++++++++++++++++++ PC time (Studio start):", current_time)
        print("is_BSI_acquisition_running?", studio.acquisitions().is_acquisition_running())
        sync_checkpoints.append(["BSI camera start", dateandtime, current_time])
    except Exception as e:
        print(f"Error in Studio acquisition: {e}")
#§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•
#§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•

def run_multiple_cameras(cam_list):
    threads = []

    #threads.append(serial_thread)
    for i, cam in enumerate(cam_list):
        cam.Init()
        set_frame_rate(cam, FPS_TARGET)
        flir_thread = threading.Thread(target=acquire_images, args=(cam, i, cam_list))
        threads.append(flir_thread)

    # Initialize Studio acquisition thread
    studio_thread = threading.Thread(target=run_studio_acquisition)
    threads.append(studio_thread)

    #The serial communication thread
    #serial_thread = threading.Thread(target=handle_serial_communication, args = (s,))
    #serial_thread.start()

    # Start all threads (they will wait for the capture_start_event)
    for thread in threads:
        thread.start()

    # All threads are ready, signal them to start acquisition
    capture_start_event.set()

    # Join threads after completion
    for thread in threads:
        thread.join()
    #serial_thread.join() #########HERE
    print("Shut down all threads")

    # Deinitialize each camera
    for cam in cam_list:
        if cam.IsStreaming():
            cam.EndAcquisition()
        cam.DeInit()
    print("End acquisition and deinitialize each camera")

    return True
#§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•
#§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•


# Function to read logs from custom circuit board after acquisition
def read_mimic_logs(s, wait_time):
    print("\nREADING MIMIC LOGS")
    play_log = ''
    frame_log = ''

    #check for DONEP! line before sending PLOG and FLOG
    donep = False
    if donep:
        received_data = ''
        while True:
            data = s.readline().decode()
            if s.in_waiting > 0:
                received_data += s.readline().decode() #CHECK READ FORMAT
                print(received_data)
                if '>DONEP!' in received_data:
                    print("'DONEP!' received.")
                    break

    print("waiting for DONEP")
    time.sleep(25)
    #time.sleep(NUM_IMAGES*(1/40)+10)
    try:
        #collect PLOG
        s.write('>PLOG!\n'.encode())
        print("•••••••••••••••••Wrote >PLOG!")
        while True:
            if s.in_waiting >= 35:
                data = s.readline().decode().replace("\n", "")
                #print("$$$$$$$$$$$$$$$$$$$$ data:", data)
                play_log += data #+ '\n'
                if '>DONER!' in data:
                    break

        #collect FLOG
        s.write('>FLOG!\n'.encode())
        print("•••••••••••••••••Wrote >FLOG!")
        while True:
            if s.in_waiting >= 50:
                data = s.readline().decode().replace("\n", "")
                #print("$$$$$$$$$$$$$$$$$$$$ data:", data)
                frame_log += data #+ '\n'
                if '>DONER!' in data:
                    break

        # Saving log files
        playlog_path = f"{NEW_FOLDER}/playlog_{current_time_label}.txt"
        framelog_path = f"{NEW_FOLDER}/framelog_{current_time_label}.txt"
        print("Writing to", playlog_path, "and", framelog_path)
        with open(playlog_path, 'w') as file:
            file.write(play_log)
        with open(framelog_path, 'w') as file:
            file.write(frame_log)

        if s and s.is_open:
            s.close()
            print("Serial port closed.")

    except serial.SerialException as e:
        print(f"Serial communication error when reading out logs: {e}")
#§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•
#§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•

def save_data_to_folder(fps_l, time_stamp_l, time_stamp_pc_l, fps_list_1, fps_list_2, sync_checkpoints):
    # Format current date and time into a string suitable for a directory name
    additional_data_folder = os.path.join(NEW_FOLDER, "additional data (fps, timestamps)")
    print("additional_data_folder path:", additional_data_folder)

    # Create the directory if it doesn't exist
    if not os.path.exists(additional_data_folder):
        os.makedirs(additional_data_folder)

    # Save fps_l to a file
    with open(os.path.join(additional_data_folder, "fps_l.txt"), 'w') as file:
        for item in fps_l:
            file.write(f"{item}\n")

    # Save time_stamp_l to a file
    with open(os.path.join(additional_data_folder, "time_stamp_l.txt"), 'w') as file:
        for item in time_stamp_l:
            file.write(f"{item}\n")

    # Save time_stamp_pc_l to a file
    with open(os.path.join(additional_data_folder, "time_stamp_pc_l.txt"), 'w') as file:
        for pair in time_stamp_pc_l:
            file.write(f"{pair[0]}, {pair[1]}, {pair[2]}, {pair[3]}\n")

    with open(os.path.join(additional_data_folder, "fps_list_1.txt"), 'w') as file:
        file.write("FPS for FLIR camera 1 based on camera timestamps\n")
        for fps in fps_list_1:
            file.write(f"{fps}\n")

    with open(os.path.join(additional_data_folder, "fps_list_2.txt"), 'w') as file:
        file.write("FPS for FLIR camera 2 based on camera timestamps\n")
        for fps in fps_list_2:
            file.write(f"{fps}\n")

    with open(os.path.join(additional_data_folder, "sync_checkpoints.txt"), 'w') as file:
        for sublist in sync_checkpoints:
            line = str(sublist) + '\n'
            file.write(line)

    #with open(os.path.join(additional_data_folder, "flir_timestep_avg_l.txt"), 'w') as file:
    #    for avg in flir_timestep_avg_l:
    #        line = str(sublist) + '\n'
    #        file.write(line)
#§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•
#§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•

def main():

    #send_start_time_to_mimic()

    result = True
    # Retrieve singleton reference to system object
    system = pyspin.System.GetInstance()
    # Get current library version
    version = system.GetLibraryVersion()
    print('Library version: %d.%d.%d.%d' % (version.major, version.minor, version.type, version.build))
    # Retrieve list of cameras from the system
    cam_list = system.GetCameras()
    print("Cam list:", cam_list)
    num_cameras = cam_list.GetSize()
    print('Number of cameras detected: %d' % num_cameras)

    # Finish if there are no cameras
    if num_cameras == 0:
        # Clear camera list before releasing system
        cam_list.Clear()
        # Release system instance
        system.ReleaseInstance()
        print('Not enough cameras!')
        input('Done! Press Enter to exit...')
        return False

    # Run example on all cameras
    print('Running example for all cameras...')
    result = run_multiple_cameras(cam_list)
    print('Example complete... \n')

    # Clear camera list before releasing system
    cam_list.Clear()
    print("Cameras cleared")

    # Release system instance
    system.ReleaseInstance()

    ########## Serial Communication with Mimic Board #########
    if readout and use_serial:
        read_mimic_logs(s, 10)
        print("Acquisition and log reading complete.")
        print('Shutdown serial port')

    #FPS measures from python time
    print("FPS (perf counter)", fps_l)
    print("STDV of FPS", np.std(np.array(fps_l[3:])))
    print("Abs mean difference", np.mean(np.abs(np.array(fps_l[3:]) - 40)))

    #Get the timestamp of FLIR using average of perf_counter
    #flir_timestep_avg_l = []
    #for pair in time_stamp_pc_l:
    #    flir_timestep_avg_l.append((pair[0]+pair[1])/2)
    #print("Avg derived timestamp for FLIR", flir_timestep_avg_l)

    ########## Calculate FPS from the list of timestamps ##########
    timestamps = time_stamp_l
    fps_list_1 = []
    fps_list_2 = []
    for i in range(2,len(timestamps)-2):
        #camera 1
        time_difference_seconds = (timestamps[i] - timestamps[i-2]) / 1e9  # Convert nanoseconds to seconds
        fps = 1 / time_difference_seconds
        fps_list_1.append(fps)
        #camera 2
        time_difference_seconds = (timestamps[i+1] - timestamps[i-1]) / 1e9  # Convert nanoseconds to seconds
        fps = 1 / time_difference_seconds
        fps_list_2.append(fps)
    # Print the calculated FPS values
    print("fps_list1", fps_list_1)
    print("fps_list2", fps_list_2)
    print("len(fps_list)", len(fps_list_1))
    print("len(timestamps)", len(timestamps))

    ########## Save all data #########
    save_data_to_folder(fps_l, time_stamp_l, time_stamp_pc_l, fps_list_1, fps_list_2, sync_checkpoints)

    input('Done! Press Enter to exit...')
    return result
#§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•
#§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•§•
if __name__ == '__main__':
    if main():
        sys.exit(0)
    else:
        sys.exit(1)

New folder path: C:\MyCameraData\FLIR_BSI_TS_FPS_2024-06-23_13-26-27 

Current date and time: 2024-06-23_13-26-27.846959
High-precision time (in nanoseconds): 22363202400
Serial port is open and configured.
Library version: 2.4.0.144
Cam list: <pyspin.PySpin.CameraList; proxy of <Swig Object of type 'Spinnaker::CameraList *' at 0x000002013FE05ED0> >
Number of cameras detected: 1
Running example for all cameras...
*** IMAGE ACQUISITION FOR CAMERA 0 ***



C:\Users\fractal_1\anaconda3\envs\my_gpu_env\lib\site-packages\pyjavaz\bridge.py:346: UserWarning: Version mistmatch between Java ZMQ server and Python client. 
Java ZMQ server version: 4.0.0
Python client expected version: 5.1.0
 To fix, update to BOTH Java and Python sides of bridge
  warnings.warn(


Camera 0 started acquiring images...
++++++++++++++++++++++++++++++++++++++++++++ Begin FLIR acquisition
Camera 0: Frame 0 captured at 29.17 FPS
Camera 0: Frame 1 captured at 40.35 FPS
++++++++++++++++++++++++++++++++++++++++++++ PC time (Studio start): 23598133300
is_BSI_acquisition_running? True
Camera 0: Frame 2 captured at 38.61 FPS
Camera 0: Frame 3 captured at 41.79 FPS
Camera 0: Frame 4 captured at 39.97 FPS
Camera 0: Frame 5 captured at 39.87 FPS
Camera 0: Frame 6 captured at 40.25 FPS
Camera 0: Frame 7 captured at 40.05 FPS
Camera 0: Frame 8 captured at 42.86 FPS
Camera 0: Frame 9 captured at 40.00 FPS
Camera 0: Frame 10 captured at 39.95 FPS
Camera 0: Frame 11 captured at 39.93 FPS
$$$$$$$$$$$$$$$$$$$$data: Sync Detected, and waiting for GO! and Timecode

Camera 0: Frame 12 captured at 39.91 FPS
$$$$$$$$$$$$$$$$$$$$data: >SYNC!

Frametime command written in acquire images: >GO!2024-06-:10

++++++++++++++++++++++++++++++++++++++++++++ Wrote GO to serial
Camera 0: Frame 13 capt

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

